In [1]:
# imports
from connection import create_spotify_oauth, get_audio_features, get_token, get_tracks, \
    tracks_to_df, audio_features_to_df, get_artist_info, artist_info_to_df, get_album_info, \
        album_info_to_df, extract_lists

import json

import pandas as pd
import numpy as np

In [2]:
# redirect uri, token and authentication variables
redirect_uri_string = 'localhost:5000/authorize/'

sp_ouath = create_spotify_oauth(redirect_uri_string)
token_info = get_token(redirect_uri_string)

In [3]:
# getting playlist items

playlist_source = '5a4unKpRA7mYi2PAgKTuNW' # playlist id for playlist

tracks = get_tracks(playlist_source, token_info) # parses Spotipy.playlist_items() to fetch playlist items
tracks_df = tracks_to_df(tracks) # extracts relevant features from messy list into a df

tracks_df.drop_duplicates(subset=['track_id'], inplace=True) # drops duplicate songs based on track ID, this is useful
# if a song gets added from single and album release; or in the case of deluxe versions of albums, etc.


In [4]:
# getting tracks' features

tracks_features = []

for track in tracks_df['track_id']:
    track_af = get_audio_features(track, token_info) # parses Spotipy.audio_features() to fetch track features
    tracks_features.extend(track_af) # extends each track's features into the empty list init'd before
    
tracks_features_df = audio_features_to_df(tracks_features) # converts nested data from API call to simple df


,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,20j7l6tFtc5lynXpbOQg1O,0.756,0.394,4,-7.321,0,0.0956,0.9450,0.037100,0.1200,0.197,70.027,127055,4
1,6dtheMr76Ks6j6wGlwyU09,0.475,0.406,6,-13.865,0,0.0503,0.1050,0.000105,0.1000,0.250,131.106,196033,4
2,03WMhty0KQ2ppU30LiyKy4,0.667,0.734,0,-9.891,1,0.3180,0.6100,0.052300,0.6410,0.145,142.968,222373,4
3,4NCBDTOYQmBmXXQEs3pUDB,0.359,0.138,8,-14.730,1,0.0439,0.9180,0.000000,0.1110,0.198,122.903,134320,4
4,5bJ1DrEM4hNCafcDd1oxHx,0.520,0.538,1,-8.109,1,0.2060,0.3670,0.001870,0.1190,0.177,141.894,345133,4
5,0HvbFmfFlcsAJ6hydqflNk,0.668,0.475,7,-7.663,1,0.0325,0.4420,0.000337,0.0997,0.586,111.993,190148,4
6,0SsOMCjOCpit4EfFpwjcFW,0.515,0.300,0,-12.084,1,0.0253,0.7170,0.062200,0.1310,0.107,82.011,202777,4
7,2LlOeW5rVcvl3QcPNPcDus,0.603,0.441,9,-8.178,1,0.0518,0.6920,0.000001,0.1050,0.167,136.137,225313,4
8,6efkcs2aUBMFKxl0cl2JWQ,0.639,0.295,0,-11.237,0,0.5440,0.5540,0.000031,0.3000,0.634,86.947,169320,4
9,56cwONDc1AtfPNXgCNb4hv,0.573,0.572,11,-8.105,0,0.0364,0.3050,0.001420,0.1520,0.371,128.847,194267,4


In [9]:
# getting tracks' artists

tracks_artists = []

for artist_id in tracks_df['artist_id']:
    for i in artist_id: # since one track can have multiple artists, loop over the list of artists to pull data for each one
        artist_info = get_artist_info(i, token_info)
        tracks_artists.append(artist_info)

artists_df = artist_info_to_df(tracks_artists) # converts nested data to df

In [12]:
# getting tracks' albums

tracks_albums = []

for album_id in tracks_df['album_id']:
    album_info = get_album_info(album_id, token_info)
    tracks_albums.append(album_info)

# with open('all_tracks_albums.json', 'w') as outfile:
#     json.dump(all_tracks_albums, outfile, indent=4)

albums_df = album_info_to_df(tracks_albums) # converts nested data to df

In [ ]:
# # preprocessing data
# merged_tracks_df = pd.concat([all_tracks_df, all_tracks_features_df], axis=1) # merging tracks' info and features

# merged_tracks_df = merged_tracks_df.loc[:, ~merged_tracks_df.columns.duplicated()] # dropping duplicate cols (track_id)

# merged_tracks_df['album_date'] = pd.to_datetime(merged_tracks_df['album_date'], format='ISO8601') # changed album_date type to datetime

# merged_tracks_df['explicit'] = merged_tracks_df['explicit'].replace({True: 1, 
#                                                                      False: 0, 
#                                                                      'nan': 0}) # encode explicit values, if nan assume not explicit



# merged_tracks_df = merged_tracks_df.drop(columns=['track_name', 'album_name', 
#                                                   'preview_url', 'album_cover_640', 
#                                                   'album_cover_300','album_cover_64']) # dropping cols irrelevant to model

